In [1]:
# importing the necessary libraries and establishing SQL Connection.
import pandas as pd
import numpy as np
import time
import collections
import mysql.connector
import math
from pandas.io import sql
import  datetime as dt
import mysql.connector as sqlcon
import os
import psycopg2 as pg
import sys
import json
import seaborn as sns
import sqlalchemy
from datetime import datetime
import utils1  
import db_config as db
from datetime import datetime,timedelta
import ast

pd.set_option("display.max_columns", None)
cnx_swiggy = mysql.connector.connect(user='shadowfax', password='shadowfax123', database= 'swiggy', host='test.prod.db.shadowfax.in')
RMS = db.credentials['RMS']
cnx_rms = sqlcon.connect(user = RMS['user'], password = RMS['password'] , host = RMS['host'],database = RMS['database'])
hl_read = db.credentials['hl_read']
cnx_hl = sqlcon.connect(user=hl_read['user'], password=hl_read['password'], host=hl_read['host'],database=hl_read['database'])
frodo = db.credentials['frodo']
cnx_frodo = pg.connect( host = frodo['host'] , user = frodo['user'] ,password = frodo['password'] ,  database = frodo['database'])
CPS= db.credentials['CPS']
cnx_cps= sqlcon.connect( host=CPS['host'], user=CPS['user'], password=CPS['password'],  database=CPS['database'])
smaug = db.credentials['Smaug']
cnx_smaug = pg.connect( host = smaug['host'] , user = smaug['user'] ,password = smaug['password'] ,  database = smaug['database'])
Ecom = db.credentials['Ecom']
cnx_ecom = sqlcon.connect(user=Ecom['user'], password=Ecom['password'], host=Ecom['host'],database=Ecom['database'])

print("Done")


Done


In [2]:
# i_date = dt.datetime(2023,8,24)

f_date = dt.datetime(2023,8,31)
s_date = f_date - dt.timedelta(days=8)
e_date= f_date + dt.timedelta(days=0)

s_date = s_date.strftime('%Y-%m-%d') + ' 18:30:00'
e_date = e_date.strftime('%Y-%m-%d') + ' 18:30:00'

print("")
print(s_date)
print(e_date)
print("")


2023-08-23 18:30:00
2023-08-31 18:30:00



In [ ]:
Ecom = db.credentials['Ecom']
cnx_ecom = sqlcon.connect(user=Ecom['user'], password=Ecom['password'], host=Ecom['host'],database=Ecom['database'])


def get_data1(s_date, e_date):
        query = '''
        select 
        dr.awb_number,
        dr.product_value,
        dr.client_id,
        h.name as hub,
        ep.code as pincode,
        eds2.action_time + interval '330' minute as picked_date,
        eds.action_time + interval '330' minute as request_date,
        dr.delivery_date + interval '330' minute as delivery_date,
        edp.client_promised_date,
        eds.remarks,
        dr.order_status,
        dr.rider_id,
        odc1.name as origin_dc,
        odc2.name as destination_dc,
        odc1.city_id as first,
        odc2.city_id as second,
        state_change_remarks
        
        from ecommerce_deliveryrequest dr
        left join ecommerce_runsheet er on dr.runsheet_id= er.id
        left join ecommerce_deliveryrequestadditionalparams edp on edp.delivery_request_id = dr.id
        left join ecommerce_deliveryrequeststatelog eds on eds.delivery_request_id = dr.id and eds.new_state = 0
        left join ecommerce_deliveryrequeststatelog eds2 on eds2.delivery_request_id = dr.id and eds2.new_state in (33,9) and eds2.old_state = 0
        left join ecommerce_hub h on h.id = dr.hub_id
        left join dc_dc_mapping_dispatchcenter odc1 on odc1.id=dr.origin_dc
        left join dc_dc_mapping_dispatchcenter odc2 on odc2.id=dr.dispatch_center
        left join ecommerce_pincode ep on ep.id = dr.customer_pincode_id
        
        where 
        eds.action_time between '{s_date}' and '{e_date}'
        and dr.client_id in (1043, 1282, 1283, 2045, 2245, 2277)
        '''.format(s_date= s_date, e_date= e_date)
        df = sql.read_sql(query,cnx_ecom)
        return df


df = get_data1(s_date,e_date)
print("DONE")

In [1]:
print('duplicate',len(df[df[['awb_number']].duplicated()]))
df = df.drop_duplicates(subset=['awb_number'], keep='first')
print('duplicate',len(df[df[['awb_number']].duplicated()]))
df[df[['awb_number']].duplicated()]
df

NameError: name 'df' is not defined

In [ ]:
forward_order_status = {
    0 :  'NEW',
    1 :  'TO_BE_ASSIGNED',
    2 :  'ASSIGNED',
    3 :  'OUT_FOR_DELIVERY',
    4 :  'DELIVERED',
    5 :  'TO_BE_RETURNED',
    6 :  'CLOSED',
    7 :  'ASSIGNED_TO_CLUSTER',
    8 :  'LOST',
    9 :  'RECEIVED_AT_DC',
    10 :  'SCHEDULED',
    11 :  'CID',
    12 :  'CANCELLED',
    13 :  'NOT_CONTACTABLE',
    14 :  'NOT_ATTEMPTED',
    15 :  'RECEIVED_AT_HUB',
    16 :  'RETURNED_TO_CLIENT',
    17 :  'RTO',
    18 :  'RTD',
    19 :  'NDR_DEL',
    20 :  'IN_MANIFEST',
    21 :  'IN_RETURN_PROCESS',
    22 :  'RTS',
    23 :  'RTOD',
    24 :  'RTSD',
    25 :  'RTOND',
    26 :  'RTSND',
    27 :  'RECEIVED_AT_PICKUP_HUB',
    28 :  'PINCODE_UPDATED',
    29 :  'ON_HOLD',
    30 :  'CLIENT_REJECTED',
    31 :  'RECEIVED_AT_VIA_HUB',
    32 :  'DELIVERED_AT_HUB',
    33 :  'RECEIVED_FROM_CLIENT_WAREHOUSE',
    34 :  'SELLER_REJECTED',
    35 :  'EWB_ON_HOLD',
    36 :  'OFD_TO_STORE',
    37 :  'RECEIVED_AT_STORE',
    38 :  'RETURN_FROM_STORE',
    39 :  'MISROUTE'
}



df['latest_status']=df.order_status.apply(lambda x: forward_order_status[x])
df


In [ ]:
df['req_date'] = df['request_date'].dt.date
df['pick_date'] = df['picked_date'].dt.date
df['deliver_date'] = df['delivery_date'].dt.date
df['client_p_date'] = df['client_promised_date'].dt.date
df['sdd_tag1'] = 'not_sdd'
df.loc[ ( (df['client_id']==1043) &(df['client_promised_date'].dt.hour ==23) & (df['client_promised_date'].dt.minute ==0) & (df['first']==df['second'])), 'sdd_tag1' ] = 'sdd'
# df['sdd_tag2'] = 'not_sdd'
# df.loc[(df['pick_date'] == df['client_p_date']),'sdd_tag2'] = 'sdd'
df

In [ ]:
df.to_csv('base.csv')

In [ ]:
df.loc[df['sdd_tag1']=='sdd']

In [ ]:
df1 = df.fillna(0)
df1['req_date'] = df1['req_date'].astype(str)
df1['deliver_date'] = df1['deliver_date'].astype(str)
df1['pick_date'] = df1['pick_date'].astype(str)
df1['client_p_date'] = df1['client_p_date'].astype(str)
pivot = pd.pivot_table(df1, index = ['req_date','pick_date','client_p_date','deliver_date','client_id','hub','pincode','origin_dc','sdd_tag1','order_status','latest_status'], values = ['awb_number'], aggfunc = {'awb_number':pd.Series.nunique}).rename(columns={'awb_number':'orders'}).reset_index()
pivot



In [ ]:
# import utils
# pivot = pivot.fillna(0)
# pivot['req_date'] = pivot['req_date'].astype(str)
# pivot['deliver_date'] = pivot['deliver_date'].astype(str)
# pivot['pick_date'] = pivot['pick_date'].astype(str)
# pivot['client_p_date'] = pivot['client_p_date'].astype(str)
# pivot.replace([np.inf, -np.inf], 0, inplace=True)
# link = 'https://docs.google.com/spreadsheets/d/1HcgavnzaHtg_P1ry81GudlEN5kUNRJKGlp1BbcE0Vyk/edit#gid=1734477288'

# ghseet_ = utils.open_gsheet_by_url(link)
# worksheet_ = utils.access_worksheet_by_name(ghseet_, 'base_data')
# utils.update_data_in_gsheet(worksheet_, pivot, start_cell='a1')



## Pendency data

In [91]:
# importing the necessary libraries and establishing SQL Connection.
import pandas as pd
import numpy as np
import time
import collections
import mysql.connector
import math
from pandas.io import sql
import  datetime as dt
import mysql.connector as sqlcon
import os
import psycopg2 as pg
import sys
import json
import seaborn as sns
import sqlalchemy
from datetime import datetime
import utils1  
import db_config as db
from datetime import datetime,timedelta
import ast

pd.set_option("display.max_columns", None)
cnx_swiggy = mysql.connector.connect(user='shadowfax', password='shadowfax123', database= 'swiggy', host='test.prod.db.shadowfax.in')
RMS = db.credentials['RMS']
cnx_rms = sqlcon.connect(user = RMS['user'], password = RMS['password'] , host = RMS['host'],database = RMS['database'])
hl_read = db.credentials['hl_read']
cnx_hl = sqlcon.connect(user=hl_read['user'], password=hl_read['password'], host=hl_read['host'],database=hl_read['database'])
frodo = db.credentials['frodo']
cnx_frodo = pg.connect( host = frodo['host'] , user = frodo['user'] ,password = frodo['password'] ,  database = frodo['database'])
CPS= db.credentials['CPS']
cnx_cps= sqlcon.connect( host=CPS['host'], user=CPS['user'], password=CPS['password'],  database=CPS['database'])
smaug = db.credentials['Smaug']
cnx_smaug = pg.connect( host = smaug['host'] , user = smaug['user'] ,password = smaug['password'] ,  database = smaug['database'])
Ecom = db.credentials['Ecom']
cnx_ecom = sqlcon.connect(user=Ecom['user'], password=Ecom['password'], host=Ecom['host'],database=Ecom['database'])

print("Done")


Done


In [100]:
s_date = pd.to_datetime('2023-06-16', dayfirst=True)
s_date = s_date.date()

print("")
print(s_date)
print("")


2023-06-16



In [101]:
Ecom = db.credentials['Ecom']
cnx_ecom = sqlcon.connect(user=Ecom['user'], password=Ecom['password'], host=Ecom['host'],database=Ecom['database'])
i_date1 = s_date


def get_db(s_date,e_date):
    query = ''' 
    
        select 
        dr.awb_number,
        dr.product_value,
        dr.client_id,
        h.name as hub,
        ep.code as pincode,
        eds2.action_time + interval '330' minute as picked_date,
        eds.action_time + interval '330' minute as request_date,
        dr.delivery_date + interval '330' minute as delivery_date,
        edp.client_promised_date,
        eds.remarks,
        dr.order_status,
        dr.rider_id,
        odc1.name as origin_dc,
        odc2.name as destination_dc,
        odc1.city_id as first,
        odc2.city_id as second,
        state_change_remarks
        from 
        ecommerce_deliveryrequest dr
        left join ecommerce_runsheet er on dr.runsheet_id= er.id
        left join ecommerce_deliveryrequestadditionalparams edp on edp.delivery_request_id = dr.id
        left join ecommerce_deliveryrequeststatelog eds on eds.delivery_request_id = dr.id and eds.new_state = 0
        left join ecommerce_deliveryrequeststatelog eds2 on eds2.delivery_request_id = dr.id and eds2.new_state in (33,9) and eds2.old_state = 0
        left join ecommerce_hub h on h.id = dr.hub_id
        left join dc_dc_mapping_dispatchcenter odc1 on odc1.id=dr.origin_dc
        left join dc_dc_mapping_dispatchcenter odc2 on odc2.id=dr.dispatch_center
        left join ecommerce_pincode ep on ep.id = dr.customer_pincode_id
        
        where 
        eds.action_time between '{s_date} 18:30:00' and '{e_date} 18:30:00'
        and dr.client_id in (1043) 
        and hour(edp.client_promised_date) = 23 
        and minute(edp.client_promised_date) = 0
        and odc1.city_id = odc2.city_id 
        and dr.order_status not in (4,23)
    
'''.format(s_date = s_date, e_date = e_date)
    df = pd.read_sql(query, cnx_ecom)
    return df



print(i_date1)
pin = pd.DataFrame()
base = pd.DataFrame()

#############################################################

for i in range(8):
    print(i)
    f_date1 = i_date1 + dt.timedelta(days=1)
    Ecom = db.credentials['Ecom']
    cnx_ecom = sqlcon.connect(user=Ecom['user'], password=Ecom['password'], host=Ecom['host'],database=Ecom['database'])

    pin = get_db(i_date1,f_date1)   
    base = base.append(pin)
    
    print(i_date1)
    i_date1 = f_date1+ dt.timedelta(days=0)
    print(f_date1)
    
############################################################


print("DONE")



2023-06-16
0
2023-06-16
2023-06-17
1
2023-06-17
2023-06-18
2
2023-06-18
2023-06-19
3
2023-06-19
2023-06-20
4
2023-06-20
2023-06-21
5
2023-06-21
2023-06-22
6
2023-06-22
2023-06-23
7
2023-06-23
2023-06-24
DONE


In [102]:
print('duplicate',len(base[base[['awb_number']].duplicated()]))
base = base.drop_duplicates(subset=['awb_number'], keep='first')
print('duplicate',len(base[base[['awb_number']].duplicated()]))
base[base[['awb_number']].duplicated()]
base

duplicate 0
duplicate 0


,awb_number,product_value,client_id,hub,pincode,picked_date,request_date,delivery_date,client_promised_date,remarks,order_status,rider_id,origin_dc,destination_dc,first,second,state_change_remarks
0,SF600939636FK,344,1043,BLR_Deepanjalinagar,560026,NaT,2023-06-17 11:58:51,None,2023-06-17 23:00:00,None,12,None,Bangalore DC,Bangalore DC,2,2,6.0
1,SF600901607FK,255,1043,BLR_Bellandur_N,560102,2023-06-17 13:53:14,2023-06-17 11:59:45,None,2023-06-17 23:00:00,None,8,None,Bangalore DC,Bangalore DC,2,2,NaN
2,SF600967636FK,170,1043,BLR_Whitefield,560066,NaT,2023-06-17 12:07:56,None,2023-06-17 23:00:00,None,12,None,Bangalore DC,Bangalore DC,2,2,6.0
3,SF600915137FK,211,1043,BLR_Whitefield,560066,NaT,2023-06-17 12:10:03,None,2023-06-17 23:00:00,None,12,None,Bangalore DC,Bangalore DC,2,2,6.0
4,SF600519004FK,242,1043,BLR_Whitefield,560066,NaT,2023-06-17 12:21:39,None,2023-06-17 23:00:00,None,12,None,Bangalore DC,Bangalore DC,2,2,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,SF600619387FK,499,1043,BLR_Deepanjalinagar,560026,NaT,2023-06-23 12:48:18,None,2023-06-23 23:00:00,None,12,None,Bangalore DC,Bangalore DC,2,2,6.0
5,SF600524927FK,952,1043,BLR_Whitefield,560087,NaT,2023-06-23 13:26:46,None,2023-06-23 23:00:00,None,12,None,Bangalore DC,Bangalore DC,2,2,6.0
0,SF600820041FK,44098,1043,BLR_Whitefield,560066,2023-06-24 14:15:04,2023-06-24 11:39:56,None,2023-06-24 23:00:00,None,8,213403.0,Bangalore DC,Bangalore DC,2,2,NaN
1,SF600766545FK,386,1043,BLR_Whitefield,560087,2023-06-24 14:28:12,2023-06-24 12:39:23,None,2023-06-24 23:00:00,None,8,216093.0,Bangalore DC,Bangalore DC,2,2,NaN


In [103]:
pendings = base.loc[~base['picked_date'].isna()]
pendings

,awb_number,product_value,client_id,hub,pincode,picked_date,request_date,delivery_date,client_promised_date,remarks,order_status,rider_id,origin_dc,destination_dc,first,second,state_change_remarks
1,SF600901607FK,255,1043,BLR_Bellandur_N,560102,2023-06-17 13:53:14,2023-06-17 11:59:45,None,2023-06-17 23:00:00,None,8,None,Bangalore DC,Bangalore DC,2,2,NaN
2,SF600516571FK,199,1043,BLR_Amblipura,560035,2023-06-18 13:27:47,2023-06-18 11:55:13,None,2023-06-18 23:00:00,None,8,NaN,Bangalore DC,Bangalore DC,2,2,NaN
4,SF601004704FK,190,1043,BLR_Kundalahalli,560037,2023-06-18 13:31:53,2023-06-18 11:58:34,None,2023-06-18 23:00:00,None,8,NaN,Bangalore DC,Bangalore DC,2,2,NaN
5,SF600580069FK,204,1043,BLR_Kundalahalli,560037,2023-06-18 13:26:41,2023-06-18 11:58:47,None,2023-06-18 23:00:00,None,8,NaN,Bangalore DC,Bangalore DC,2,2,NaN
8,SF599932627FK,1499,1043,BOM_KoparKhairane,400705,2023-06-19 06:18:46,2023-06-18 18:53:23,None,2023-07-04 23:00:00,None,8,186090.0,Mumbai DC,Mumbai DC,4,4,NaN
6,SF600998792FK,315,1043,BLR_Kundalahalli,560037,2023-06-19 13:18:55,2023-06-19 11:52:53,None,2023-06-19 23:00:00,None,8,None,Bangalore DC,Bangalore DC,2,2,NaN
7,SF600916785FK,2709,1043,BLR_Amblipura,560035,2023-06-19 13:29:42,2023-06-19 11:54:25,None,2023-06-19 23:00:00,None,8,None,Bangalore DC,Bangalore DC,2,2,NaN
10,SF600952293FK,102,1043,BLR_Amblipura,560035,2023-06-19 13:21:23,2023-06-19 12:24:48,None,2023-06-19 23:00:00,None,8,None,Bangalore DC,Bangalore DC,2,2,NaN
4,SF600535722FK,299,1043,BLR_Kadugodi,560067,2023-06-20 13:26:30,2023-06-20 11:31:19,None,2023-06-20 23:00:00,None,8,NaN,Bangalore DC,Bangalore DC,2,2,NaN
8,SF600528569FK,1109,1043,BOM_KoparKhairane,400705,2023-06-20 19:13:32,2023-06-20 13:11:23,None,2023-07-05 23:00:00,None,8,115542.0,Mumbai DC,Mumbai DC,4,4,NaN


In [104]:
forward_order_status = {
    0 :  'NEW',
    1 :  'TO_BE_ASSIGNED',
    2 :  'ASSIGNED',
    3 :  'OUT_FOR_DELIVERY',
    4 :  'DELIVERED',
    5 :  'TO_BE_RETURNED',
    6 :  'CLOSED',
    7 :  'ASSIGNED_TO_CLUSTER',
    8 :  'LOST',
    9 :  'RECEIVED_AT_DC',
    10 :  'SCHEDULED',
    11 :  'CID',
    12 :  'CANCELLED',
    13 :  'NOT_CONTACTABLE',
    14 :  'NOT_ATTEMPTED',
    15 :  'RECEIVED_AT_HUB',
    16 :  'RETURNED_TO_CLIENT',
    17 :  'RTO',
    18 :  'RTD',
    19 :  'NDR_DEL',
    20 :  'IN_MANIFEST',
    21 :  'IN_RETURN_PROCESS',
    22 :  'RTS',
    23 :  'RTOD',
    24 :  'RTSD',
    25 :  'RTOND',
    26 :  'RTSND',
    27 :  'RECEIVED_AT_PICKUP_HUB',
    28 :  'PINCODE_UPDATED',
    29 :  'ON_HOLD',
    30 :  'CLIENT_REJECTED',
    31 :  'RECEIVED_AT_VIA_HUB',
    32 :  'DELIVERED_AT_HUB',
    33 :  'RECEIVED_FROM_CLIENT_WAREHOUSE',
    34 :  'SELLER_REJECTED',
    35 :  'EWB_ON_HOLD',
    36 :  'OFD_TO_STORE',
    37 :  'RECEIVED_AT_STORE',
    38 :  'RETURN_FROM_STORE',
    39 :  'MISROUTE'
}



pendings['latest_status'] = pendings.order_status.apply(lambda x: forward_order_status[x])
pendings


<ipython-input-104-d32fea7f246a>:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pendings['latest_status'] = pendings.order_status.apply(lambda x: forward_order_status[x])


,awb_number,product_value,client_id,hub,pincode,picked_date,request_date,delivery_date,client_promised_date,remarks,order_status,rider_id,origin_dc,destination_dc,first,second,state_change_remarks,latest_status
1,SF600901607FK,255,1043,BLR_Bellandur_N,560102,2023-06-17 13:53:14,2023-06-17 11:59:45,None,2023-06-17 23:00:00,None,8,None,Bangalore DC,Bangalore DC,2,2,NaN,LOST
2,SF600516571FK,199,1043,BLR_Amblipura,560035,2023-06-18 13:27:47,2023-06-18 11:55:13,None,2023-06-18 23:00:00,None,8,NaN,Bangalore DC,Bangalore DC,2,2,NaN,LOST
4,SF601004704FK,190,1043,BLR_Kundalahalli,560037,2023-06-18 13:31:53,2023-06-18 11:58:34,None,2023-06-18 23:00:00,None,8,NaN,Bangalore DC,Bangalore DC,2,2,NaN,LOST
5,SF600580069FK,204,1043,BLR_Kundalahalli,560037,2023-06-18 13:26:41,2023-06-18 11:58:47,None,2023-06-18 23:00:00,None,8,NaN,Bangalore DC,Bangalore DC,2,2,NaN,LOST
8,SF599932627FK,1499,1043,BOM_KoparKhairane,400705,2023-06-19 06:18:46,2023-06-18 18:53:23,None,2023-07-04 23:00:00,None,8,186090.0,Mumbai DC,Mumbai DC,4,4,NaN,LOST
6,SF600998792FK,315,1043,BLR_Kundalahalli,560037,2023-06-19 13:18:55,2023-06-19 11:52:53,None,2023-06-19 23:00:00,None,8,None,Bangalore DC,Bangalore DC,2,2,NaN,LOST
7,SF600916785FK,2709,1043,BLR_Amblipura,560035,2023-06-19 13:29:42,2023-06-19 11:54:25,None,2023-06-19 23:00:00,None,8,None,Bangalore DC,Bangalore DC,2,2,NaN,LOST
10,SF600952293FK,102,1043,BLR_Amblipura,560035,2023-06-19 13:21:23,2023-06-19 12:24:48,None,2023-06-19 23:00:00,None,8,None,Bangalore DC,Bangalore DC,2,2,NaN,LOST
4,SF600535722FK,299,1043,BLR_Kadugodi,560067,2023-06-20 13:26:30,2023-06-20 11:31:19,None,2023-06-20 23:00:00,None,8,NaN,Bangalore DC,Bangalore DC,2,2,NaN,LOST
8,SF600528569FK,1109,1043,BOM_KoparKhairane,400705,2023-06-20 19:13:32,2023-06-20 13:11:23,None,2023-07-05 23:00:00,None,8,115542.0,Mumbai DC,Mumbai DC,4,4,NaN,LOST


In [106]:
pendings['req_date'] = pendings['request_date'].dt.date
pendings['pick_date'] = pendings['picked_date']#.dt.date
pendings['client_p_date'] = pendings['client_promised_date'].dt.date
pendings = pendings[['awb_number','product_value','client_id','hub','pincode','req_date','pick_date','client_p_date','order_status','rider_id','origin_dc','destination_dc','latest_status']]




KeyError: 'request_date'

In [107]:
pendings#.to_csv('d1.csv')

,awb_number,product_value,client_id,hub,pincode,req_date,pick_date,client_p_date,order_status,rider_id,origin_dc,destination_dc,latest_status
1,SF600901607FK,255,1043,BLR_Bellandur_N,560102,2023-06-17,2023-06-17 13:53:14,2023-06-17,8,None,Bangalore DC,Bangalore DC,LOST
2,SF600516571FK,199,1043,BLR_Amblipura,560035,2023-06-18,2023-06-18 13:27:47,2023-06-18,8,NaN,Bangalore DC,Bangalore DC,LOST
4,SF601004704FK,190,1043,BLR_Kundalahalli,560037,2023-06-18,2023-06-18 13:31:53,2023-06-18,8,NaN,Bangalore DC,Bangalore DC,LOST
5,SF600580069FK,204,1043,BLR_Kundalahalli,560037,2023-06-18,2023-06-18 13:26:41,2023-06-18,8,NaN,Bangalore DC,Bangalore DC,LOST
8,SF599932627FK,1499,1043,BOM_KoparKhairane,400705,2023-06-18,2023-06-19 06:18:46,2023-07-04,8,186090.0,Mumbai DC,Mumbai DC,LOST
6,SF600998792FK,315,1043,BLR_Kundalahalli,560037,2023-06-19,2023-06-19 13:18:55,2023-06-19,8,None,Bangalore DC,Bangalore DC,LOST
7,SF600916785FK,2709,1043,BLR_Amblipura,560035,2023-06-19,2023-06-19 13:29:42,2023-06-19,8,None,Bangalore DC,Bangalore DC,LOST
10,SF600952293FK,102,1043,BLR_Amblipura,560035,2023-06-19,2023-06-19 13:21:23,2023-06-19,8,None,Bangalore DC,Bangalore DC,LOST
4,SF600535722FK,299,1043,BLR_Kadugodi,560067,2023-06-20,2023-06-20 13:26:30,2023-06-20,8,NaN,Bangalore DC,Bangalore DC,LOST
8,SF600528569FK,1109,1043,BOM_KoparKhairane,400705,2023-06-20,2023-06-20 19:13:32,2023-07-05,8,115542.0,Mumbai DC,Mumbai DC,LOST


In [108]:
import utils
pendings = pendings.fillna(0)
pendings['req_date'] = pendings['req_date'].astype(str)
pendings['pick_date'] = pendings['pick_date'].astype(str)
pendings['client_p_date'] = pendings['client_p_date'].astype(str)
pendings.replace([np.inf, -np.inf], 0, inplace=True)
link = 'https://docs.google.com/spreadsheets/d/1HcgavnzaHtg_P1ry81GudlEN5kUNRJKGlp1BbcE0Vyk/edit#gid=1734477288'

ghseet_ = utils.open_gsheet_by_url(link)
worksheet_ = utils.access_worksheet_by_name(ghseet_, 'pendency')
utils.update_data_in_gsheet(worksheet_, pendings, start_cell='a4646')



True